In [1]:
import pandas

df = pandas.read_parquet("./data/complete.parquet")


In [52]:
import itertools
def get_vocab(x):
    return x.split(" ")






print(len(set((df[df["language"] == "EN"]["estilo_x"]))))
print(len(df[df["language"] == "EN"]["estilo_x"]))

print(df[df["language"] == "EN"]["sourceLen"].mean())
print("Inglês : ", len(set(list(itertools.chain(*df[df["language"] == "EN"]["texto_x"].apply(lambda x: get_vocab(x)))))))

print("--------")
print(len(set((df[df["language"] == "ES"]["estilo_x"]))))
print(len(df[df["language"] == "ES"]["estilo_x"]))
print(df[df["language"] == "ES"]["sourceLen"].mean())
print("Espanhol : ", len(set(list(itertools.chain(*df[df["language"] == "ES"]["texto_x"].apply(lambda x: get_vocab(x)))))))

print("--------")
print(len(set((df[df["language"] == "PT"]["estilo_x"]))))
print(len(df[df["language"] == "PT"]["estilo_x"]))
print(df[df["language"] == "PT"]["sourceLen"].mean())
print("Portugues : ", len(set(list(itertools.chain(*df[df["language"] == "PT"]["texto_x"].apply(lambda x: get_vocab(x)))))))

print("--------")
print(len(set((df[df["language"] == "IT"]["estilo_x"]))))
print(len(df[df["language"] == "IT"]["estilo_x"]))
print(df[df["language"] == "IT"]["sourceLen"].mean())
print("Italiano : ", len(set(list(itertools.chain(*df[df["language"] == "IT"]["texto_x"].apply(lambda x: get_vocab(x)))))))


print("--------")
print(len(set((df[df["language"] == "FR"]["estilo_x"]))))
print(len(df[df["language"] == "FR"]["estilo_x"]))
print(df[df["language"] == "FR"]["sourceLen"].mean())
print("Frances : ", len(set(list(itertools.chain(*df[df["language"] == "FR"]["texto_x"].apply(lambda x: get_vocab(x)))))))

31
646838
33.43115277704773
Inglês :  70130
--------
13
287684
29.016705830007925
Espanhol :  85873
--------
5
111490
27.220010763297157
Portugues :  53481
--------
4
84071
27.04393905151598
Italiano :  47447
--------
3
73716
32.26888328178414
Frances :  35089


In [2]:
from nltk import ngrams


def tokenize(text, nGram=3):
    text = text.split(" ")
    grams = []
    for i in range(1, nGram):
        i_grams = [
            " ".join(gram)
            for gram in ngrams(text, i)
        ]
        grams.extend(i_grams)
        
    return grams

def getNgramOverlap(x, nGram):
  overlaps = []
  for h, r in zip([x["texto_x"]], [x["texto_y"]]):
    if (h == "") or (r == ""):
      overlaps.append(1.0)
      continue
    a = tokenize(h, nGram)
    b = tokenize(r, nGram)

    if len(a) >= len(b) and len(a) > 0:
      overlaps.append(len(set(a) & set(b))/len(a))
    elif len(b) >= len(a) and len(b) > 0:
      overlaps.append(len(set(a) & set(b))/len(b))
    elif len(a) == 0 or len(b) == 0:
      overlaps.append(0)

  return overlaps[0]





In [1]:
ARRUMAR AS OUTRAS MÉTRICAS

# ttr - OK
# x-formal
# diferença de tamanho entre versiculos (Origem e Alvo) - OK
# Kendall-tau  - OK
# sobreposição (unigrama) - OK


SyntaxError: invalid syntax (775573991.py, line 1)

In [3]:
df_formal   = df[df["formal/informal_x"] == "formal"]
df_informal = df[df["formal/informal_x"] == "informal"]

df_formal_informal = pandas.merge(df_formal, df_informal, on=["livro", "capitulo", "versiculo", "language"], how="inner").drop(["estilo_x_x", "VERSAO_x", "forma_x_x", "linguagem_x_x", "estilo_x_y",
                                                                                                            "VERSAO_y", "forma_x_y", "linguagem_x_y"], axis=1).rename(columns={"texto_x_x": "texto_x", "texto_x_y": "texto_y", 
                                                                                                                                                                               "formal/informal_x_y": "formal/informal_y",
                                                                                                                                                                               "formal/informal_x_x": "formal/informal_x",
                                                                                                                                                                               "sourceLen_x": "sourceLen", "sourceLen_y": "targetLen"})


In [ ]:
progress_apply

In [3]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TextClassificationPipeline
import torch


tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased", )
model = AutoModelForSequenceClassification.from_pretrained("./model_mbert")

pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, device=0, batch_size=256)


from datasets import Dataset
from transformers.pipelines.pt_utils import KeyDataset



def get_proba(a):

    list_informal = []
    list_formal   = []

    for dicts in a:
        for dict in dicts:
            if dict["label"] == "LABEL_0":
                list_informal.append(dict["score"])
            else:
                list_formal.append(dict["score"])
    return (list_informal, list_formal)


from tqdm import tqdm




c:\Users\pablo\anaconda3\envs\torch\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


0.49999999955968183

In [11]:
from scipy.stats import kendalltau
import string
import math
import re 

def get_ttr(x):
    tokens = x.split(" ")
    return (len(set(tokens)) / len(tokens)) * 100
def get_kendall_tau(x):
    x1 = normalize_answer(x["texto_x"])
    x2 = normalize_answer(x["texto_y"])

    x1_tokens = x1.split()
    x2_tokens = x2.split()

    for x1_index, tok in enumerate(x1_tokens):
        try:
            x2_index = x2_tokens.index(tok)
            x1_tokens[x1_index] = "<match-found>-{:d}".format(x1_index + 1)
            x2_tokens[x2_index] = "<match-found>-{:d}".format(x1_index + 1)
        except ValueError:
            pass

    common_seq_x1 = [int(x1_tok_flag.split("-")[-1]) for x1_tok_flag in x1_tokens if x1_tok_flag.startswith("<match-found>")]
    common_seq_x2 = [int(x2_tok_flag.split("-")[-1]) for x2_tok_flag in x2_tokens if x2_tok_flag.startswith("<match-found>")]

    assert len(common_seq_x1) == len(common_seq_x2)

    ktd = kendalltau(common_seq_x1, common_seq_x2).correlation
    anomaly = False

    if math.isnan(ktd):
        ktd = -1.0
        anomaly = True

    return ktd


def normalize_answer(s):
    """Lower text and remove punctuation and extra whitespace."""


    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_punc(lower(s)))

def modulus(x):
    if x >= 0:
        return x
    elif x < 0: 
        return -(x)

In [19]:
data_set = Dataset.from_dict({"text_x":  df_formal_informal[df_formal_informal["language"] == "EN"]["texto_x"].tolist(), "text_y": df_formal_informal[df_formal_informal["language"] == "EN"]["texto_y"].tolist()})
from torch.utils.data import DataLoader

dataloader = DataLoader(data_set, batch_size=256)

preds_x = []

for out in tqdm(dataloader):
    
    preds_x.extend(pipe(out["text_x"], return_all_scores=True,  max_length=512))

preds_y = []    

for out in tqdm(dataloader):
    
    preds_y.extend(pipe(out["text_y"], return_all_scores=True,  max_length=512))


probasX = get_proba(preds_x)
probasY = get_proba(preds_y)

import numpy as np

print("formality_source: ", np.average(probasX))
print("formality_target: ", np.average(probasY))

0.27651131494308373

In [13]:

df_formal   = df[df["formal/informal_x"] == "formal"]
df_informal = df[df["formal/informal_x"] == "informal"]

df_formal_informal = pandas.merge(df_formal, df_informal, on=["livro", "capitulo", "versiculo", "language"], how="inner").drop(["estilo_x_x", "VERSAO_x", "forma_x_x", "linguagem_x_x", "estilo_x_y",
                                                                                                            "VERSAO_y", "forma_x_y", "linguagem_x_y"], axis=1).rename(columns={"texto_x_x": "texto_x", "texto_x_y": "texto_y", 
                                                                                                                                                                               "formal/informal_x_y": "formal/informal_y",
                                                                                                                                                                               "formal/informal_x_x": "formal/informal_x",
                                                                                                                                                                               "sourceLen_x": "sourceLen", "sourceLen_y": "targetLen"})

from tqdm.auto import tqdm
tqdm.pandas()
print(df_formal_informal[df_formal_informal["language"] == "EN"].progress_apply(lambda x: get_kendall_tau(x), axis=1).mean())

100%|██████████| 2358195/2358195 [06:42<00:00, 5860.55it/s]

0.6309980297976011


In [14]:

df_formal   = df[df["formal/informal_x"] == "formal"]
df_informal = df[df["formal/informal_x"] == "informal"]

df_formal_informal = pandas.merge(df_formal, df_informal, on=["livro", "capitulo", "versiculo", "language"], how="inner").drop(["estilo_x_x", "VERSAO_x", "forma_x_x", "linguagem_x_x", "estilo_x_y",
                                                                                                            "VERSAO_y", "forma_x_y", "linguagem_x_y"], axis=1).rename(columns={"texto_x_x": "texto_x", "texto_x_y": "texto_y", 
                                                                                                                                                                               "formal/informal_x_y": "formal/informal_y",
                                                                                                                                                                               "formal/informal_x_x": "formal/informal_x",
                                                                                                                                                                               "sourceLen_x": "sourceLen", "sourceLen_y": "targetLen"})

print("English")
print("3-gram: ", df_formal_informal[df_formal_informal["language"] == "EN"].apply(lambda x: getNgramOverlap(x, 3), axis=1).mean())
print("tamanho: ", df_formal_informal[df_formal_informal["language"] == "EN"].shape[0])
print("sourceLen: ", df_formal_informal[df_formal_informal["language"] == "EN"]["sourceLen"].mean())
print("targetLen: ", df_formal_informal[df_formal_informal["language"] == "EN"]["targetLen"].mean())
print("diferença: ", (df_formal_informal[df_formal_informal["language"] == "EN"]["sourceLen"] - df_formal_informal[df_formal_informal["language"] == "EN"]["targetLen"]).apply(lambda x: modulus(x)).mean())
print("ttr_source: ", df_formal_informal[df_formal_informal["language"] == "EN"]["texto_x"].apply(lambda x: get_ttr(x)).mean())
print("ttr_target: ", df_formal_informal[df_formal_informal["language"] == "EN"]["texto_y"].apply(lambda x: get_ttr(x)).mean())
print("kendall: ", df_formal_informal[df_formal_informal["language"] == "EN"].progress_apply(lambda x: get_kendall_tau(x), axis=1).mean())






print("-------------")

print("spanish")
print("3-gram: ", df_formal_informal[df_formal_informal["language"] == "ES"].apply(lambda x: getNgramOverlap(x, 3), axis=1).mean())
print("tamanho: ", df_formal_informal[df_formal_informal["language"] == "ES"].shape[0])
print("sourceLen: ", df_formal_informal[df_formal_informal["language"] == "ES"]["sourceLen"].mean())
print("targetLen: ", df_formal_informal[df_formal_informal["language"] == "ES"]["targetLen"].mean())
print("diferença: ", (df_formal_informal[df_formal_informal["language"] == "ES"]["sourceLen"] - df_formal_informal[df_formal_informal["language"] == "ES"]["targetLen"]).apply(lambda x: modulus(x)).mean())
print("ttr_source: ", df_formal_informal[df_formal_informal["language"] == "ES"]["texto_x"].apply(lambda x: get_ttr(x)).mean())
print("ttr_target: ", df_formal_informal[df_formal_informal["language"] == "ES"]["texto_y"].apply(lambda x: get_ttr(x)).mean())
print("kendall: ", df_formal_informal[df_formal_informal["language"] == "ES"].progress_apply(lambda x: get_kendall_tau(x), axis=1).mean())



print("-------------")

print("Portuguese")
print("3-gram: ", df_formal_informal[df_formal_informal["language"] == "PT"].apply(lambda x: getNgramOverlap(x, 3), axis=1).mean())
print("tamanho: ", df_formal_informal[df_formal_informal["language"] == "PT"].shape[0])
print("sourceLen: ", df_formal_informal[df_formal_informal["language"] == "PT"]["sourceLen"].mean())
print("targetLen: ", df_formal_informal[df_formal_informal["language"] == "PT"]["targetLen"].mean())
print("diferença: ", (df_formal_informal[df_formal_informal["language"] == "PT"]["sourceLen"] - df_formal_informal[df_formal_informal["language"] == "PT"]["targetLen"]).apply(lambda x: modulus(x)).mean())
print("ttr_source: ", df_formal_informal[df_formal_informal["language"] == "PT"]["texto_x"].apply(lambda x: get_ttr(x)).mean())
print("ttr_target: ", df_formal_informal[df_formal_informal["language"] == "PT"]["texto_y"].apply(lambda x: get_ttr(x)).mean())
print("kendall: ", df_formal_informal[df_formal_informal["language"] == "PT"].progress_apply(lambda x: get_kendall_tau(x), axis=1).mean())




print("-------------")

print("French")
print("3-gram: ", df_formal_informal[df_formal_informal["language"] == "FR"].apply(lambda x: getNgramOverlap(x, 3), axis=1).mean())
print("tamanho: ", df_formal_informal[df_formal_informal["language"] == "FR"].shape[0])
print("sourceLen: ", df_formal_informal[df_formal_informal["language"] == "FR"]["sourceLen"].mean())
print("targetLen: ", df_formal_informal[df_formal_informal["language"] == "FR"]["targetLen"].mean())
print("diferença: ", (df_formal_informal[df_formal_informal["language"] == "FR"]["sourceLen"] - df_formal_informal[df_formal_informal["language"] == "FR"]["targetLen"]).apply(lambda x: modulus(x)).mean())
print("ttr_source: ", df_formal_informal[df_formal_informal["language"] == "FR"]["texto_x"].apply(lambda x: get_ttr(x)).mean())
print("ttr_target: ", df_formal_informal[df_formal_informal["language"] == "FR"]["texto_y"].apply(lambda x: get_ttr(x)).mean())
print("kendall: ", df_formal_informal[df_formal_informal["language"] == "FR"].progress_apply(lambda x: get_kendall_tau(x), axis=1).mean())




print("-------------")

print("Italiano")
print("3-gram: ", df_formal_informal[df_formal_informal["language"] == "IT"].apply(lambda x: getNgramOverlap(x, 3), axis=1).mean())
print("tamanho: ", df_formal_informal[df_formal_informal["language"] == "IT"].shape[0])
print("sourceLen: ", df_formal_informal[df_formal_informal["language"] == "IT"]["sourceLen"].mean())
print("targetLen: ", df_formal_informal[df_formal_informal["language"] == "IT"]["targetLen"].mean())
print("diferença: ", (df_formal_informal[df_formal_informal["language"] == "IT"]["sourceLen"] - df_formal_informal[df_formal_informal["language"] == "IT"]["targetLen"]).apply(lambda x: modulus(x)).mean())
print("ttr_source: ", df_formal_informal[df_formal_informal["language"] == "IT"]["texto_x"].apply(lambda x: get_ttr(x)).mean())
print("ttr_target: ", df_formal_informal[df_formal_informal["language"] == "IT"]["texto_y"].apply(lambda x: get_ttr(x)).mean())
print("kendall: ", df_formal_informal[df_formal_informal["language"] == "IT"].progress_apply(lambda x: get_kendall_tau(x), axis=1).mean())







English
3-gram:  0.25684361660729643
tamanho:  2358195
sourceLen:  33.60680647698769
targetLen:  30.464953491971613
diferença:  8.25639864387805
ttr_source:  81.46112965033961
ttr_target:  83.64502439665871


100%|██████████| 2358195/2358195 [06:44<00:00, 5827.03it/s]


kendall:  0.6309980297976011
-------------
spanish
3-gram:  0.2814773631605753
tamanho:  717880
sourceLen:  29.91112860088037
targetLen:  27.87728311138352
diferença:  6.408518136735945
ttr_source:  84.67258974424347
ttr_target:  86.90092562955311


100%|██████████| 717880/717880 [01:59<00:00, 6015.20it/s]


kendall:  0.6411774812856231
-------------
Portuguese
3-gram:  0.27651131494308373
tamanho:  82342
sourceLen:  26.93297466663428
targetLen:  28.678572296033614
diferença:  4.9460785504359865
ttr_source:  86.30104165882648
ttr_target:  86.78456607658833


...

kendall:  0.6292981192708266
-------------
French
3-gram:  0.33274791333966053
tamanho:  43834
sourceLen:  30.869074234612402
targetLen:  35.57430305242506
diferença:  7.188324131952366
ttr_source:  83.35920467660084
ttr_target:  83.32682617110191


100%|██████████| 43834/43834 [00:08<00:00, 5477.04it/s]


kendall:  0.7293913893469365
-------------
Italiano
3-gram:  0.27388901021327416
tamanho:  16172
sourceLen:  24.951026465495918
targetLen:  28.427652733118972
diferença:  6.238807815978234
ttr_source:  89.10627304708233
ttr_target:  88.83199685095266


100%|██████████| 16172/16172 [00:02<00:00, 6361.43it/s]

kendall:  0.6732066354784958


,livro,capitulo,versiculo,texto_x,sourceLen,language,linguagem_x,texto_y,targetLen,linguagem_y
0,Gênesis,1,18,"e para governar o dia e a noite , e para fazer...",27,PT,arcaica,para governarem o dia e a noite e para separar...,26,moderna
1,Gênesis,1,19,E foi a tarde e a manhã : o dia quarto .,12,PT,arcaica,"A noite passou , e veio a manhã . Esse foi o q...",15,moderna
2,Gênesis,1,20,E disse Deus : Produzam as águas abundantement...,25,PT,arcaica,Depois Deus disse : — Que as águas fiquem chei...,28,moderna
3,Gênesis,1,21,"E Deus criou as grandes baleias , e todo répti...",40,PT,arcaica,"Assim Deus criou os grandes monstros do mar , ...",45,moderna
4,Gênesis,1,22,"E Deus os abençoou , dizendo : Frutificai , e ...",27,PT,arcaica,Ele abençoou os seres vivos do mar e disse : —...,31,moderna
...,...,...,...,...,...,...,...,...,...,...
3218418,Apocalisse,22,7,"« Ecco , sto per venire . Beato chi custodisce...",19,IT,moderna,"Il Signore dice : « Ecco , io vengo presto ! B...",22,moderna
3218419,Apocalisse,22,8,"Io , * Giovanni , sono quello che ha udito e v...",38,IT,moderna,"Io , Giovanni , ho visto queste cose . Dopo di...",29,moderna
3218420,Apocalisse,22,16,"Io , Gesú , ho mandato il mio angelo per attes...",36,IT,moderna,"Io , Gesù , vi ho mandato il mio angelo per di...",35,moderna
3218421,Apocalisse,22,17,( C ) Lo Spirito e la * sposa dicono : « Vieni...,42,IT,moderna,E lo Spirito e la sposa dellʼAgnello dicono : ...,38,moderna


In [16]:
df_formal   = df[df["linguagem_x"] == "arcaica"]
df_informal = df[df["linguagem_x"] == "moderna"]

df_formal_informal = pandas.merge(df_formal, df_informal, on=["livro", "capitulo", "versiculo", "language"], how="inner").drop(["estilo_x_x", "VERSAO_x", "forma_x_x", "formal/informal_x_x", "estilo_x_y",
                                                                                                            "VERSAO_y", "forma_x_y", "formal/informal_x_y"], axis=1).rename(columns={"texto_x_x": "texto_x", "texto_x_y": "texto_y", 
                                                                                                                                                                               "linguagem_x_y": "linguagem_y",
                                                                                                                                                                               "linguagem_x_x": "linguagem_x",
                                                                                                                                                                               "sourceLen_x": "sourceLen", "sourceLen_y": "targetLen"})

print("English")
print("3-gram: ", df_formal_informal[df_formal_informal["language"] == "EN"].apply(lambda x: getNgramOverlap(x, 3), axis=1).mean())
print("tamanho: ", df_formal_informal[df_formal_informal["language"] == "EN"].shape[0])
print("sourceLen: ", df_formal_informal[df_formal_informal["language"] == "EN"]["sourceLen"].mean())
print("targetLen: ", df_formal_informal[df_formal_informal["language"] == "EN"]["targetLen"].mean())
print("diferença: ", (df_formal_informal[df_formal_informal["language"] == "EN"]["sourceLen"] - df_formal_informal[df_formal_informal["language"] == "EN"]["targetLen"]).apply(lambda x: modulus(x)).mean())
print("ttr_source: ", df_formal_informal[df_formal_informal["language"] == "EN"]["texto_x"].apply(lambda x: get_ttr(x)).mean())
print("ttr_target: ", df_formal_informal[df_formal_informal["language"] == "EN"]["texto_y"].apply(lambda x: get_ttr(x)).mean())
print("kendall: ", df_formal_informal[df_formal_informal["language"] == "EN"].progress_apply(lambda x: get_kendall_tau(x), axis=1).mean())




print("-------------")

print("spanish")
print("3-gram: ", df_formal_informal[df_formal_informal["language"] == "ES"].apply(lambda x: getNgramOverlap(x, 3), axis=1).mean())
print("tamanho: ", df_formal_informal[df_formal_informal["language"] == "ES"].shape[0])
print("sourceLen: ", df_formal_informal[df_formal_informal["language"] == "ES"]["sourceLen"].mean())
print("targetLen: ", df_formal_informal[df_formal_informal["language"] == "ES"]["targetLen"].mean())
print("diferença: ", (df_formal_informal[df_formal_informal["language"] == "ES"]["sourceLen"] - df_formal_informal[df_formal_informal["language"] == "ES"]["targetLen"]).apply(lambda x: modulus(x)).mean())
print("ttr_source: ", df_formal_informal[df_formal_informal["language"] == "ES"]["texto_x"].apply(lambda x: get_ttr(x)).mean())
print("ttr_target: ", df_formal_informal[df_formal_informal["language"] == "ES"]["texto_y"].apply(lambda x: get_ttr(x)).mean())
print("kendall: ", df_formal_informal[df_formal_informal["language"] == "ES"].progress_apply(lambda x: get_kendall_tau(x), axis=1).mean())



print("-------------")

print("Portuguese")
print("3-gram: ", df_formal_informal[df_formal_informal["language"] == "PT"].apply(lambda x: getNgramOverlap(x, 3), axis=1).mean())
print("tamanho: ", df_formal_informal[df_formal_informal["language"] == "PT"].shape[0])
print("sourceLen: ", df_formal_informal[df_formal_informal["language"] == "PT"]["sourceLen"].mean())
print("targetLen: ", df_formal_informal[df_formal_informal["language"] == "PT"]["targetLen"].mean())
print("diferença: ", (df_formal_informal[df_formal_informal["language"] == "PT"]["sourceLen"] - df_formal_informal[df_formal_informal["language"] == "PT"]["targetLen"]).apply(lambda x: modulus(x)).mean())
print("ttr_source: ", df_formal_informal[df_formal_informal["language"] == "PT"]["texto_x"].apply(lambda x: get_ttr(x)).mean())
print("ttr_target: ", df_formal_informal[df_formal_informal["language"] == "PT"]["texto_y"].apply(lambda x: get_ttr(x)).mean())
print("kendall: ", df_formal_informal[df_formal_informal["language"] == "PT"].progress_apply(lambda x: get_kendall_tau(x), axis=1).mean())




print("-------------")

print("French")
print("3-gram: ", df_formal_informal[df_formal_informal["language"] == "FR"].apply(lambda x: getNgramOverlap(x, 3), axis=1).mean())
print("tamanho: ", df_formal_informal[df_formal_informal["language"] == "FR"].shape[0])
print("sourceLen: ", df_formal_informal[df_formal_informal["language"] == "FR"]["sourceLen"].mean())
print("targetLen: ", df_formal_informal[df_formal_informal["language"] == "FR"]["targetLen"].mean())
print("diferença: ", (df_formal_informal[df_formal_informal["language"] == "FR"]["sourceLen"] - df_formal_informal[df_formal_informal["language"] == "FR"]["targetLen"]).apply(lambda x: modulus(x)).mean())
print("ttr_source: ", df_formal_informal[df_formal_informal["language"] == "FR"]["texto_x"].apply(lambda x: get_ttr(x)).mean())
print("ttr_target: ", df_formal_informal[df_formal_informal["language"] == "FR"]["texto_y"].apply(lambda x: get_ttr(x)).mean())
print("kendall: ", df_formal_informal[df_formal_informal["language"] == "FR"].progress_apply(lambda x: get_kendall_tau(x), axis=1).mean())




print("-------------")

print("Italiano")
print("3-gram: ", df_formal_informal[df_formal_informal["language"] == "IT"].apply(lambda x: getNgramOverlap(x, 3), axis=1).mean())
print("tamanho: ", df_formal_informal[df_formal_informal["language"] == "IT"].shape[0])
print("sourceLen: ", df_formal_informal[df_formal_informal["language"] == "IT"]["sourceLen"].mean())
print("targetLen: ", df_formal_informal[df_formal_informal["language"] == "IT"]["targetLen"].mean())
print("diferença: ", (df_formal_informal[df_formal_informal["language"] == "IT"]["sourceLen"] - df_formal_informal[df_formal_informal["language"] == "IT"]["targetLen"]).apply(lambda x: modulus(x)).mean())
print("ttr_source: ", df_formal_informal[df_formal_informal["language"] == "IT"]["texto_x"].apply(lambda x: get_ttr(x)).mean())
print("ttr_target: ", df_formal_informal[df_formal_informal["language"] == "IT"]["texto_y"].apply(lambda x: get_ttr(x)).mean())
print("kendall: ", df_formal_informal[df_formal_informal["language"] == "IT"].progress_apply(lambda x: get_kendall_tau(x), axis=1).mean())




English
3-gram:  0.31482014604161207
tamanho:  2494087
sourceLen:  32.32388164486644
targetLen:  33.82381929740222
diferença:  7.9871443939205005
ttr_source:  80.35878665212681
ttr_target:  82.0666342758966


100%|██████████| 2494087/2494087 [07:25<00:00, 5603.26it/s] 


kendall:  0.72607142738709
-------------
spanish
3-gram:  0.35038181170144994
tamanho:  221222
sourceLen:  33.83691495420889
targetLen:  28.511879469492186
diferença:  6.914976810624621
ttr_source:  83.26719865142074
ttr_target:  85.77000005857792


100%|██████████| 221222/221222 [00:45<00:00, 4906.26it/s]


kendall:  0.7548290488706166
-------------
Portuguese
3-gram:  0.2774112948778335
tamanho:  97333
sourceLen:  27.26286048924825
targetLen:  27.10199007530848
diferença:  4.576885537279237
ttr_source:  84.43333575979467
ttr_target:  87.34660124498974


...

kendall:  0.6477461134990186
-------------
French
3-gram:  0.33274791333966053
tamanho:  43834
sourceLen:  30.869074234612402
targetLen:  35.57430305242506
diferença:  7.188324131952366
ttr_source:  83.35920467660084
ttr_target:  83.32682617110191


100%|██████████| 43834/43834 [00:08<00:00, 5066.33it/s]


kendall:  0.7293913893469365
-------------
Italiano
3-gram:  0.4663175320012529
tamanho:  54556
sourceLen:  27.150799178825427
targetLen:  27.018953002419533
diferença:  3.2786311313146124
ttr_source:  87.63315109314738
ttr_target:  87.96531659058834


100%|██████████| 54556/54556 [00:09<00:00, 5812.45it/s]

kendall:  0.8249850745048948


In [104]:
pandas.merge(df_formal, df_informal, on=["livro", "capitulo", "versiculo", "language"], how="inner").drop(["estilo_x_x", "VERSAO_x", "linguagem_x_x", "formal/informal_x_x", "estilo_x_y",
                                                                                                            "VERSAO_y", "linguagem_x_y", "formal/informal_x_y"], axis=1).rename(columns={"texto_x_x": "texto_x", "texto_x_y": "texto_y", 
                                                                                                                                                                               "forma_x_x": "forma_x",
                                                                                                                                                                               "forma_x_y": "forma_y",
                                                                                                                                                                               "sourceLen_x": "sourceLen", "sourceLen_y": "targetLen"})

,livro,capitulo,versiculo,texto_x,sourceLen,language,forma_x,texto_y,targetLen,forma_y
0,Gênesis,1,18,"e para governar o dia e a noite , e para fazer...",27,PT,literal,para governarem o dia e a noite e para separar...,26,dinâmica
1,Gênesis,1,18,"e para governar o dia e a noite , e para fazer...",27,PT,literal,"governar o dia e a noite , e separar a luz das...",21,dinâmica
2,Gênesis,1,19,E foi a tarde e a manhã : o dia quarto .,12,PT,literal,"A noite passou , e veio a manhã . Esse foi o q...",15,dinâmica
3,Gênesis,1,19,E foi a tarde e a manhã : o dia quarto .,12,PT,literal,Passaram-se a tarde e a manhã ; esse foi o qua...,13,dinâmica
4,Gênesis,1,20,E disse Deus : Produzam as águas abundantement...,25,PT,literal,Depois Deus disse : — Que as águas fiquem chei...,28,dinâmica
...,...,...,...,...,...,...,...,...,...,...
3173853,Apocalisse,22,7,"« Ecco , sto per venire . Beato chi custodisce...",19,IT,literal,"Il Signore dice : « Ecco , io vengo presto ! B...",22,dinâmica
3173854,Apocalisse,22,8,"Io , * Giovanni , sono quello che ha udito e v...",38,IT,literal,"Io , Giovanni , ho visto queste cose . Dopo di...",29,dinâmica
3173855,Apocalisse,22,16,"Io , Gesú , ho mandato il mio angelo per attes...",36,IT,literal,"Io , Gesù , vi ho mandato il mio angelo per di...",35,dinâmica
3173856,Apocalisse,22,17,( C ) Lo Spirito e la * sposa dicono : « Vieni...,42,IT,literal,E lo Spirito e la sposa dellʼAgnello dicono : ...,38,dinâmica


In [15]:
df_formal   = df[df["forma_x"] == "literal"]
df_informal = df[df["forma_x"] == "dinâmica"]

df_formal_informal = pandas.merge(df_formal, df_informal, on=["livro", "capitulo", "versiculo", "language"], how="inner").drop(["estilo_x_x", "VERSAO_x", "linguagem_x_x", "formal/informal_x_x", "estilo_x_y",
                                                                                                            "VERSAO_y", "linguagem_x_y", "formal/informal_x_y"], axis=1).rename(columns={"texto_x_x": "texto_x", "texto_x_y": "texto_y", 
                                                                                                                                                                               "forma_x_x": "forma_x",
                                                                                                                                                                               "forma_x_y": "forma_y",
                                                                                                                                                                               "sourceLen_x": "sourceLen", "sourceLen_y": "targetLen"})

print("English")
print("3-gram: ", df_formal_informal[df_formal_informal["language"] == "EN"].apply(lambda x: getNgramOverlap(x, 3), axis=1).mean())
print("tamanho: ", df_formal_informal[df_formal_informal["language"] == "EN"].shape[0])
print("sourceLen: ", df_formal_informal[df_formal_informal["language"] == "EN"]["sourceLen"].mean())
print("targetLen: ", df_formal_informal[df_formal_informal["language"] == "EN"]["targetLen"].mean())
print("diferença: ", (df_formal_informal[df_formal_informal["language"] == "EN"]["sourceLen"] - df_formal_informal[df_formal_informal["language"] == "EN"]["targetLen"]).apply(lambda x: modulus(x)).mean())
print("ttr_source: ", df_formal_informal[df_formal_informal["language"] == "EN"]["texto_x"].apply(lambda x: get_ttr(x)).mean())
print("ttr_target: ", df_formal_informal[df_formal_informal["language"] == "EN"]["texto_y"].apply(lambda x: get_ttr(x)).mean())
print("kendall: ", df_formal_informal[df_formal_informal["language"] == "EN"].progress_apply(lambda x: get_kendall_tau(x), axis=1).mean())



print("-------------")

print("spanish")
print("3-gram: ", df_formal_informal[df_formal_informal["language"] == "ES"].apply(lambda x: getNgramOverlap(x, 3), axis=1).mean())
print("tamanho: ", df_formal_informal[df_formal_informal["language"] == "ES"].shape[0])
print("sourceLen: ", df_formal_informal[df_formal_informal["language"] == "ES"]["sourceLen"].mean())
print("targetLen: ", df_formal_informal[df_formal_informal["language"] == "ES"]["targetLen"].mean())
print("diferença: ", (df_formal_informal[df_formal_informal["language"] == "ES"]["sourceLen"] - df_formal_informal[df_formal_informal["language"] == "ES"]["targetLen"]).apply(lambda x: modulus(x)).mean())
print("ttr_source: ", df_formal_informal[df_formal_informal["language"] == "ES"]["texto_x"].apply(lambda x: get_ttr(x)).mean())
print("ttr_target: ", df_formal_informal[df_formal_informal["language"] == "ES"]["texto_y"].apply(lambda x: get_ttr(x)).mean())
print("kendall: ", df_formal_informal[df_formal_informal["language"] == "ES"].progress_apply(lambda x: get_kendall_tau(x), axis=1).mean())


print("-------------")

print("Portuguese")
print("3-gram: ", df_formal_informal[df_formal_informal["language"] == "PT"].apply(lambda x: getNgramOverlap(x, 3), axis=1).mean())
print("tamanho: ", df_formal_informal[df_formal_informal["language"] == "PT"].shape[0])
print("sourceLen: ", df_formal_informal[df_formal_informal["language"] == "PT"]["sourceLen"].mean())
print("targetLen: ", df_formal_informal[df_formal_informal["language"] == "PT"]["targetLen"].mean())
print("diferença: ", (df_formal_informal[df_formal_informal["language"] == "PT"]["sourceLen"] - df_formal_informal[df_formal_informal["language"] == "PT"]["targetLen"]).apply(lambda x: modulus(x)).mean())
print("ttr_source: ", df_formal_informal[df_formal_informal["language"] == "PT"]["texto_x"].apply(lambda x: get_ttr(x)).mean())
print("ttr_target: ", df_formal_informal[df_formal_informal["language"] == "PT"]["texto_y"].apply(lambda x: get_ttr(x)).mean())
print("kendall: ", df_formal_informal[df_formal_informal["language"] == "PT"].progress_apply(lambda x: get_kendall_tau(x), axis=1).mean())



print("-------------")

print("French")
print("3-gram: ", df_formal_informal[df_formal_informal["language"] == "FR"].apply(lambda x: getNgramOverlap(x, 3), axis=1).mean())
print("tamanho: ", df_formal_informal[df_formal_informal["language"] == "FR"].shape[0])
print("sourceLen: ", df_formal_informal[df_formal_informal["language"] == "FR"]["sourceLen"].mean())
print("targetLen: ", df_formal_informal[df_formal_informal["language"] == "FR"]["targetLen"].mean())
print("diferença: ", (df_formal_informal[df_formal_informal["language"] == "FR"]["sourceLen"] - df_formal_informal[df_formal_informal["language"] == "FR"]["targetLen"]).apply(lambda x: modulus(x)).mean())
print("ttr_source: ", df_formal_informal[df_formal_informal["language"] == "FR"]["texto_x"].apply(lambda x: get_ttr(x)).mean())
print("ttr_target: ", df_formal_informal[df_formal_informal["language"] == "FR"]["texto_y"].apply(lambda x: get_ttr(x)).mean())
print("kendall: ", df_formal_informal[df_formal_informal["language"] == "FR"].progress_apply(lambda x: get_kendall_tau(x), axis=1).mean())




print("-------------")

print("Italiano")
print("3-gram: ", df_formal_informal[df_formal_informal["language"] == "IT"].apply(lambda x: getNgramOverlap(x, 3), axis=1).mean())
print("tamanho: ", df_formal_informal[df_formal_informal["language"] == "IT"].shape[0])
print("sourceLen: ", df_formal_informal[df_formal_informal["language"] == "IT"]["sourceLen"].mean())
print("targetLen: ", df_formal_informal[df_formal_informal["language"] == "IT"]["targetLen"].mean())
print("diferença: ", (df_formal_informal[df_formal_informal["language"] == "IT"]["sourceLen"] - df_formal_informal[df_formal_informal["language"] == "IT"]["targetLen"]).apply(lambda x: modulus(x)).mean())
print("ttr_source: ", df_formal_informal[df_formal_informal["language"] == "IT"]["texto_x"].apply(lambda x: get_ttr(x)).mean())
print("ttr_target: ", df_formal_informal[df_formal_informal["language"] == "IT"]["texto_y"].apply(lambda x: get_ttr(x)).mean())
print("kendall: ", df_formal_informal[df_formal_informal["language"] == "IT"].progress_apply(lambda x: get_kendall_tau(x), axis=1).mean())




English
3-gram:  0.28719826048922836
tamanho:  2744561
sourceLen:  33.61126169176054
targetLen:  31.88786002570174
diferença:  7.8410197477848005
ttr_source:  80.52710147278582
ttr_target:  82.97547176725253


100%|██████████| 2744561/2744561 [08:18<00:00, 5504.47it/s]


kendall:  0.697397456786838
-------------
spanish
3-gram:  0.3869472877747436
tamanho:  272067
sourceLen:  28.926007196756682
targetLen:  31.084001367310258
diferença:  5.709200307277252
ttr_source:  84.58085418716409
ttr_target:  84.85080891552366


100%|██████████| 272067/272067 [00:49<00:00, 5514.89it/s]


kendall:  0.7718581152468188
-------------
Portuguese
3-gram:  0.2774112948778335
tamanho:  97333
sourceLen:  27.26286048924825
targetLen:  27.10199007530848
diferença:  4.576885537279237
ttr_source:  84.43333575979467
ttr_target:  87.34660124498974


...

kendall:  0.6477461134990186
-------------
French
3-gram:  0.48691364482557054
tamanho:  43725
sourceLen:  30.196043453401945
targetLen:  34.30072041166381
diferença:  5.366655231560892
ttr_source:  84.03624277171808
ttr_target:  82.92660782036907


100%|██████████| 43725/43725 [00:07<00:00, 5556.53it/s]


kendall:  0.8447943979367291
-------------
Italiano
3-gram:  0.27388901021327416
tamanho:  16172
sourceLen:  24.951026465495918
targetLen:  28.427652733118972
diferença:  6.238807815978234
ttr_source:  89.10627304708233
ttr_target:  88.83199685095266


100%|██████████| 16172/16172 [00:02<00:00, 5701.91it/s]

kendall:  0.6732066354784958


In [ ]:

podemos fazer um plot 3 ou 4?

a ultima dimensão poderia ser o mean sentence bert 


Preciosamos achar as transferencias que fazem sentido. Por exemplo, dinamica moderna e informal PARA literal arica e formal. Será uma tripla (tipo_tradução, linguagem, formal/informal)


Encontrar os tipos de relações criadas

forma_tradução_origem -> forma_tradução_destino -> linguagem -> formal/informal

https://github.com/ostwalprasad/PythonMultiDimensionalPlots/blob/master/src/5D.py

levantar métricas para arcaico e não arcaico - identificar o vocab em comum (em termos de porcentagem) e de pois analisar as palavras que nao sao comuns. Outra abordagem pode ser olhar o top 10 do que nao é comum de cada uma das linguas e ver se são palavras arcaicas. 

formal e informal (classificador, métricas nilc)
